# Homework 2 Question 2 

## 1. Question Analysis & Solution Designs 

### 1.1 Pre-process emails

##### Parsing Raw Data

All given email files are RFC 2822-based message documents. We can use a python built-in library `email` to parse the messages, and extract text contents. 

##### Removing HTML tags

For some of the messages, the body of is in HTML format. We need to transform them into a equivalent structured plaintext. One way is use a regular expression to match each "<>" pair and remove them. But this way is not so accurate and may remove non-tag pairs or miss some html tags. Another way is to use a FSM to parse the document and transform accordingly. `html2text` is a 3-rd party library written by 'Aaron Swartz' which is doing the job well. And I'm using his work to extract texts from html documents.

##### Implementation

Please see file `email_processor.py` for detailed implementation.

### 1.2 Build Data Set 

##### Feature Extraction

First, we need to extract features from texts we just pre-processed. As requested in the question, we should use two methods respectively :

- Word Count : use `CountVectorizer` 
- Tf-Idf : use `TfidfTransformer`

##### Shuffle

Second, to apply 5-fold cross validation effectively, we'd better shuffle the dataset first.

### 1.3 K-fold cross validation 

`sklearn.cross_validation.KFold(n, n_fold)` method will return indexes to split the data set.

### 1.4 Train models & score

Use `<model>.fit(X_train, y_train)` to train each model. 

Use `<classifer>.predict(X_test)` to predict test data.

Use `sklearn.metrics.<score_method>(y_test, y_predict)` to calculate score. 

## 2. Code Files & How To Run

The code files is arranged in the following structure: 

```
hw2
  |-- hw2q2
  |     |-- email_processor.py  (pre-process implementation)
  |     |-- util.py             (detail function defs used by main) 
  |     |-- html2text.py        (3-rd party lib to process html)
  |     |-- spam_filter.py      (main script)
  |-- spamassasin               (data folder) 
```

To run the code, first change directory to `hw2` folder, the execute: 
```Bash
python3 hw2q2/spam_filter.py
```

## 3. Main Code (spam_filter.py)

```Python
import os
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
import logging
import traceback
from sklearn import cross_validation
from sklearn.cross_validation import train_test_split as train_test_split
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import precision_score, f1_score, recall_score
from sklearn.utils import shuffle
from collections import OrderedDict


from util import iter_files, pre_process_email, word_count, tfidf, train_model, load_features


def experiment(counts, Y, isTfIdf=False):
    '''
        Conduct an experiment
            @param counts: word counts
            @param Y: lables 
            @param isTfIdf: use tf-idf or not.
        
        Each experiment will train 5 models as requested. 
        Then use 5-fold cross-validation method to split the data set.
        For each splited data set, 
            1. train the model
            2. collect scores
        Then output the requested values. 
    '''
    if isTfIdf:
        counts = tfidf(counts)
    models = {
        "1. Gaussian Naive Bayes" : GaussianNB(),
        "2. Logistic Regression L2, C=1": LogisticRegression(C=1.0, penalty='l2'),
        "3. Logistic Regression L2, C=0.5": LogisticRegression(C=0.5, penalty='l2'),
        "4. Logistic Regression L1, C=1": LogisticRegression(C=1, penalty='l1'),
        "5. Logistic Regression L1, C=0.5": LogisticRegression(C=0.5, penalty='l1'),
    }
    score_methods = {
        "precision:" : precision_score,
        "recall:": recall_score,
        "f1_score:": f1_score
    }
    orderedModels = OrderedDict(sorted(models.items(), key=lambda t: t[0]))
    for config, model in orderedModels.items():
        print("- " * 10)
        print(config)
        kf_5 = cross_validation.KFold(len(Y), n_folds=5)
        scores = {}
        for train_idx, test_idx in kf_5:
            X_train, X_test = counts[train_idx], counts[test_idx]
            y_train, y_test = Y[train_idx], Y[test_idx]
            clf = train_model(X_train.toarray(), y_train, model)
            y_predict = clf.predict(X_test.toarray())
            for item, method in score_methods.items():
                score = method(y_test, y_predict)
                if item not in scores:
                    scores[item] = [score]
                else:
                    scores[item].append(score)
        for item, score_list in scores.items():
            print("    average", item, np.mean(score_list))
            print("    std", item, np.std(score_list))


def main(corpus_base_folder):
    spam_folder = os.path.sep.join([corpus_base_folder, "spam"])
    ham_folder = os.path.sep.join([corpus_base_folder, "ham"])
    counts, Y  = load_features(
        [spam_folder, ham_folder],
        pre_process_email,
        word_count,
        [1, -1])
    counts, Y = shuffle(counts, Y, random_state=40)
    print("Using word count:")
    experiment(counts, Y, isTfIdf=False)
    print()
    print("= " * 20)
    print()
    print("Using TF-IDF:")
    experiment(counts, Y, isTfIdf=True)
```

## 4. Result 

Execution result: 
```
Using word count:
- - - - - - - - - - 
1. Gaussian Naive Bayes
    average recall: 0.778157466676
    std recall: 0.0265749149059
    average precision: 0.977554080464
    std precision: 0.0230984826665
    average f1_score: 0.865952756092
    std f1_score: 0.01226531551
- - - - - - - - - - 
2. Logistic Regression L2, C=1
    average recall: 0.912288821645
    std recall: 0.0242480881995
    average precision: 0.989697641139
    std precision: 0.00900620097033
    average f1_score: 0.949188743673
    std f1_score: 0.0124216314426
- - - - - - - - - - 
3. Logistic Regression L2, C=0.5
    average recall: 0.906832498886
    std recall: 0.0248247346833
    average precision: 0.989637145072
    std precision: 0.00905349998163
    average f1_score: 0.946186179968
    std f1_score: 0.0129628858437
- - - - - - - - - - 
4. Logistic Regression L1, C=1
    average recall: 0.918858151305
    std recall: 0.0295254914675
    average precision: 0.970055398956
    std precision: 0.013240941984
    average f1_score: 0.943546074249
    std f1_score: 0.0188891034469
- - - - - - - - - - 
5. Logistic Regression L1, C=0.5
    average recall: 0.90566025402
    std recall: 0.0289049636861
    average precision: 0.972262597742
    std precision: 0.0112666810491
    average f1_score: 0.93739278817
    std f1_score: 0.0132572289671

= = = = = = = = = = = = = = = = = = = = 

Using TF-IDF:
- - - - - - - - - - 
1. Gaussian Naive Bayes
    average recall: 0.785731506192
    std recall: 0.0438602169706
    average precision: 0.955281038565
    std precision: 0.0339682312612
    average f1_score: 0.861276667049
    std f1_score: 0.0282112398114
- - - - - - - - - - 
2. Logistic Regression L2, C=1
    average recall: 0.907150571531
    std recall: 0.0102048390634
    average precision: 0.998230088496
    std precision: 0.00353982300885
    average f1_score: 0.950463535151
    std f1_score: 0.00416039063932
- - - - - - - - - - 
3. Logistic Regression L2, C=0.5
    average recall: 0.907150571531
    std recall: 0.0102048390634
    average precision: 0.998230088496
    std precision: 0.00353982300885
    average f1_score: 0.950463535151
    std f1_score: 0.00416039063932
- - - - - - - - - - 
4. Logistic Regression L1, C=1
    average recall: 0.919142638404
    std recall: 0.0190066949001
    average precision: 0.987421744922
    std precision: 0.012747040162
    average f1_score: 0.951931588752
    std f1_score: 0.0122983819359
- - - - - - - - - - 
5. Logistic Regression L1, C=0.5
    average recall: 0.925007924271
    std recall: 0.0143772245334
    average precision: 0.987310345741
    std precision: 0.00810302493776
    average f1_score: 0.955046663372
    std f1_score: 0.00705433463702
```